In [ ]:
import cv2
import numpy as np
import math

In [ ]:
def canny(img):
    if img is None:
        cap.release()
        cv2.destroyAllWindows()
        exit()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = 5
    blur = cv2.GaussianBlur(gray, (kernel, kernel), 0)
    # Adjust Canny edge detection thresholds
    canny = cv2.Canny(blur, 30, 120)  # Lower the thresholds
    return canny
# Adjust 'make_points' to reduce offset
'''def make_points(image, line):
    slope, intercept = line
    if slope == 0:  # Avoid division by zero
        slope = 0.1
    y1 = int(image.shape[0])  # bottom of the image
    y2 = int(y1 * 3.0 / 5)  # slightly lower than the middle
    # Recalculate points based on slope and intercept with tighter bounds
    x1 = max(0, min(int((y1 - intercept) / slope), image.shape[1]))
    x2 = max(0, min(int((y2 - intercept) / slope), image.shape[1]))
    return [[x1, y1, x2, y2]]'''
def make_points(image, line):
    slope, intercept = line
    if slope == 0:  # Avoid division by zero
        slope = 0.1

    y1 = int(image.shape[0])  # Start 85% from the top of the image
    y2 = int(image.shape[0] * 0.80)  # End at 70% from the top of the image

    # Recalculate points based on slope and intercept
    x1 = max(0, min(int((y1 - intercept) / slope), image.shape[1]))
    x2 = max(0, min(int((y2 - intercept) / slope), image.shape[1]))

    return [[x1, y1, x2, y2]]

In [ ]:
def region_of_interest(canny):
    height = canny.shape[0]
    width = canny.shape[1]
    # Adjust the mask to cover more area for detecting distant lanes
    mask = np.zeros_like(canny)
    triangle = np.array([[
        (100, height),  # Left bottom corner, increased width range
        (width // 2, height // 2),  # Top center, capture more of the road ahead
        (width - 100, height)  # Right bottom corner
    ]], np.int32)
    cv2.fillPoly(mask, triangle, 255)
    masked_image = cv2.bitwise_and(canny, mask)
    return masked_image

In [ ]:
def region_of_interest(canny):
    height, width = canny.shape[:2]

    left_offset = int(width * 0.2)  # 10% from the left
    right_offset = int(width * 0.3) # 10% from the right
    top_height_ratio = 0.5          # Place the top of the triangle at 50% of the height

    triangle = np.array([[
        (left_offset, height),               # Left bottom corner
        (width // 2, int(height * top_height_ratio)),  # Top center
        (width - right_offset, height)       # Right bottom corner
    ]], np.int32)

    mask = np.zeros_like(canny)
    cv2.fillPoly(mask, triangle, 255)
    masked_image = cv2.bitwise_and(canny, mask)
    return masked_image


In [ ]:
def region_of_interest(img,polygon):
    '''mask = np.zeros_like(img)
    x1, y1 = polygon[0]
    x2, y2 = polygon[1]
    x3, y3 = polygon[2]
    x4, y4 = polygon[3]
    m1 = (y2-y1)/(x2-x1)
    m2 = (y3-y2)/(x3-x2)
    m3 = (y4-y3)/(x4-x3)
    m4 = (y4-y1)/(x4-x1)
    b1 = y1 - m1*x1
    b2 = y2 - m2*x2
    b3 = y3 - m3*x3
    b4 = y4 - m4*x4

    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if i>=m1*j+b1 and i>=m2*j+b2 and i>=m3*j+b3 and i<=m4*j+b4:
                mask[i][j] = 1'''
    polygon = np.array([[
    (0, height),
    (width, height),
    (width//2, height//2)
]], np.int32)


    # Fill the polygon in the mask
    cv2.fillPoly(mask, polygon, 255)
    '''masked_img = np.multiply(img, mask)'''
    masked_img = np.multiply(mask, img)
    return masked_img

In [ ]:
def houghLines(cropped_canny):
    # Tune minLineLength and maxLineGap for better lane detection
    return cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 50,
                           np.array([]), minLineLength=20, maxLineGap=5)
# Overlay lane lines on original frame
def add_weighted(frame, line_image):
    return cv2.addWeighted(frame, 0.8, line_image, 1, 1)

# Function to draw the detected lines on a blank image

  # Function to draw the detected lines on a blank image
def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            if line is not None: # Added check to skip None values
                for x1, y1, x2, y2 in line:
                    cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 10)  # Changed color for clearer visualization
    return line_image


In [ ]:
prev_left_line = None
prev_right_line = None
def average_slope_intercept(image, lines):
    global prev_left_line, prev_right_line
    left_fit = []
    right_fit = []
    if lines is None:
        return [prev_left_line, prev_right_line]

    # Filter lines with extreme slopes
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:  # Avoid division by zero
                continue
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - (slope * x1)

            # Filter out extreme slopes (too vertical or horizontal)
            if abs(slope) < 0.5 or abs(slope) > 2:  # Tune this range based on video
                continue

            if slope < 0:  # Negative slope => left lane
                left_fit.append((slope, intercept))
            else:  # Positive slope => right lane
                right_fit.append((slope, intercept))

    if left_fit:
        left_fit_average = np.average(left_fit, axis=0)
        left_line = make_points(image, left_fit_average)
        prev_left_line = left_line
    else:
        left_line = prev_left_line

    if right_fit:
        right_fit_average = np.average(right_fit, axis=0)
        right_line = make_points(image, right_fit_average)
        prev_right_line = right_line
    else:
        right_line = prev_right_line

    return [left_line, right_line] if left_line and right_line else None

In [ ]:
# Main function for video processing
cap = cv2.VideoCapture("/content/1stinput.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (frame_width, frame_height))

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("End of video or error reading frame.")
        break

    # Process each frame
    canny_image = canny(frame)
    height, width = canny_image.shape
    polygon = [(int(width*0.1), height), (int(width*0.45), int(height*0.6)), (int(width*0.55), int(height*0.6)), (int(0.95*width), height)]
    cropped_canny = region_of_interest(canny_image)
    lines = houghLines(cropped_canny)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = add_weighted(frame, line_image)

    # Save the output
    out.write(combo_image)
cap.release()
out.release()
cv2.destroyAllWindows()

End of video or error reading frame.
